<a href="https://colab.research.google.com/github/samuel0922/TEAM_SSAMJI/blob/main/%ED%94%8C%EC%A0%9D%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B%EC%A4%80%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd
!ls
%cd train

/content
drive  sample_data
[Errno 2] No such file or directory: 'train'
/content


In [3]:
# split-folders 모듈 설치

!pip install split-folders[full]



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import glob
import os
import splitfolders as split
import shutil as sh 

---
# Dataset 작업 내용을 진행하기 위한 진행순서 정리해 봄
---
* 데이터셋에 관해 생각해 볼 것
> * 데이터셋을 train, validation, test 데이터셋으로 나누어 구성하는 것은 overfit 발생을 최소화하기 위함임
> * 코드 내에서 train 데이터셋을 분리해서 사용할 수 있는데, 추천 코드에서는 물리적으로 분할된 데이터셋을 사용했음
> * 기존 코드를 헷갈리지 않고 최대한 잘 활용하기 위해 데이터셋의 구조를 추천코드에서 사용하는 데이터셋의 구조와 같게 맞추는 것으로 방향을 정함

## Part I. 개요
<br>
* 마스크 디텍션에서 사용하는 데이터셋의 구조는

            FaceMaskDataset
                ㅏimages
                    - .png
                ㄴannotations
                    - .xml
<br>
    인데, 코드 내에서 testset을 분리해줘서 다음과 같다

            FaceMaskDataset
                ㅏimages
                    - .png
                ㅏannotations
                    - .xml
                ㅏtest_images
                    - .png
                ㄴtest_annotations
<br>
* 농작물병해 디텍션에서 제공하는 데이터셋 구조는

        PjtDataset
            ㅏtrain
                ㅏ100000
                    ㅏ100000.jpg
                    ㅏ100000.json
                    ㄴ100000.csv
            ㄴtest
                ㅏ100000
                    ㅏ100000.jpg
                    ㄴ100000.csv

<bt>
    로 되어 있기 때문에, 두 데이터셋의 다른 점은 다음과 같다.

        1. test 데이터셋의 폴더에는 json 파일이 없다.
        2. 데이터셋의 디렉토리 구조(폴더/파일 경로)가 다르고, 이를
        3. csv파일이 제공되는데 이번 프로젝트에선 불필요하다
        4. annotation파일이 xml 대신 json을 사용하고
        5. test 데이터셋의 데이터가 너무 많다. 
            -> 처음부터 이 부분을 고려해 처리했으면 좋았을 것이다
<br>
    => 다음과 같이 전처리 해줘서 진행하기로 한다.
        1. test 데이터셋은 사용하지 않고, train 데이터셋을 나누어 train과 validation 데이터셋으로 구성한다.

        2. 디렉토리 구조(폴더, 파일)를 마스크디텍션 디렉토리 구조와 같게 맞춰준다 
        3. csv 파일은 사용하지 않는다
        4. xml 대신 json에서 annotation 정보를 parsing해서 사용한다
         
        5. train 데이터셋을 나눠서 train set과 test set으로 해준다.

## Part II. 처리 프로세스(알고리즘)

1. train 데이터셋 처리

        폴더 아래의 폴더를 읽어서 그 안에 있는 파일 중 
            jpg파일이 없으면 
                jpg 파일명을 no_images 리스트에 넣어주고 다음 파일로

            jpg 파일이 있으면
                jpg 파일을 images폴더로 copy해 줌

            json파일이 없으면
                json 파일명은 no_jsones 리스트에 넣어주고 다음 파일로
            json파일이 있으면 annotation폴더로 copy

            images 폴더의 갯수 출력
            train_no_images 리스트의 길이 출력

            annotation 폴더의 갯수 출력
            train_no_jsones 리스트의 길이 출력

2. test 데이터셋 처리 => 애초에 test 데이터셋을 그래도 활요하면 된다고 생각했지만, test 데이터셋에는 json 파일이 없기 때문에 이 데이터셋을 위의 train 데이터셋에서 취한 방식과 같게 처리해 test 데이터셋으로 사용하려고 하면 inference 에서 오류가 발생하게 됨.=> train 데이터셋을 나누어서 train셋과 test셋으로 나누고 하부 디렉토리 구조를 1과 같은 방식으로 맞춰주느 것으로 변경함.

        test 폴더 아래의 폴더를 읽어서 
            jpg파일이 없으면 
                jpg 파일명을 no_images 리스트에 넣어주고 다음 파일로

            jpg 파일이 있으면
                jpg 파일을 images폴더로 copy해 줌

            json파일이 없으면
                json 파일명은 no_jsones 리스트에 넣어주고 다음 파일로
            json파일이 있으면 annotation폴더로 copy

            images 폴더의 갯수 출력
            test_no_images 리스트의 길이 출력

            annotation 폴더의 갯수 출력
            test_no_jsones 리스트의 길이 출력

3. 주의 => 요 부분은 뒤에서 따로 해주는 걸로
* 코드 내에서 파일명의 경로 설정 시에 폴더명에 .jpg 또는 .json 
확장자를 붙여서 사용하게 만들었기 때문에, 폴더만 있고 그 안에 
jpg파일이나 json파일이 없는 경우에 파일을 못 찾는 오류가 발생함.

* 위의 과정 중에 json파일이나 jpg파일이 없는 경우는 리스트로 만들긴
했지만 이와 관련한 처리를 하지 않았기 때문에 발생되는 오류임.

=> 애초에 이 작업을 한꺼번에 진행할 수도 있고(추후에 데이터셋 작업을
새롭게 한다면...), 또는 만들어진 디렉토리 구조에서 파일이 없는 경우에 대한 처리를 해도 되는데, 일단 후자의 방법으로 처리해 준다.

        train/images 파일을 모두 읽어서 리스트 imgs_list에 저장한다.
        train/annotations 파일을 모두 읽어서 리스트 annots_list 에 저장한다

        for imgs_list로의 항목을 하나(img파일명) 읽어서
            annots_list의 항목 중에 있으면 다음으로
            annots_list의 항목 중에 없으면 삭제 

        for annots_list의 항목을 하나(annot파일명) 읽어서
            imgs_list의 항목 중에 있으면 다음으로
            imgs_list의 항목 중에 없으면 삭제



In [15]:
'''
요 코드가 작동하지 않는 이유는???
=> train 폴더 아래에 파일이 있는게 아니고 하위폴더가 있기 때문에 
split-folders()가 동작하지 않는 것

# split-folders를 사용하기 위해 FarmDataset 폴더를 만들고 아래로 class 폴더를 만들어 
# train.zip 파일을 압축 풀어 데이터셋을 준비함 
src_path = '/content/drive/MyDrive/aiffelthon/PjtDataset/train'
tgt_path = '/content/drive/MyDrive/aiffelthon/PjtDataset/FarmDataset'

# train dataset 폴더를 train/val/test로 8:1:1 나눠줌 : split-folders()
split.ratio(src_path, output=tgt_path, seed=1337, ratio=(.8, .1, .1), group_prefix=3 )
'''

Copying files: 0 files [00:01, ? files/s]


In [ ]:
train_datasets = glob.glob('*')
print(train_datasets)

['58063', '58067', '58075', '58079', '58092', '58096', '58098', '58111', '58122', '58134', '58146', '58148', '58157', '58166', '58182', '58211', '58223', '58185', '58232', '58235', '58242', '58245', '58255', '58257', '58263', '58269', '58273', '58276', '58287', '58291', '58322', '58329', '58334', '58342', '58338', '58350', '58375', '58401', '58403', '58418', '58364', '58421', '58382', '58424', '58439', '58446', '58464', '58413', '58470', '58475', '58483', '58422', '58489', '58492', '58501', '58505', '58514', '58520', '58537', '58550', '58551', '58564', '58572', '58590', '58614', '58630', '58599', '58677', '58678', '58649', '58679', '58687', '58691', '58698', '58706', '58716', '58720', '58726', '58734', '58735', '58724', '58765', '58781', '58820', '58837', '58841', '58822', '58844', '58856', '58868', '58849', '58880', '58887', '58860', '58890', '58894', '58915', '58938', '58950', '58963', '58971', '58977', '59011', '59030', '59085', '59097', '59104', '59106', '59123', '59126', '59138', 

In [ ]:
print('dataset folder : ', len(train_datasets))

no_images = []
no_jsones = []

cnt = 0

for i, train_data in enumerate(train_datasets):
    image = f'{train_data}'+'/'+f'{train_data}.jpg'
    print('image : ',image)
    if os.path.isfile(image):
        print('test1')
        sh.copy(image, './images/')
    else:
        print('test2')
        no_images.append(image)
        len(no_images)

    cnt += 1
    print('cnt : ',cnt)

print('no_images_list : ',no_images)
print('no_images_list : ',no_jsones)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
image :  18755/18755.jpg
test1
cnt :  4104
image :  18787/18787.jpg
test1
cnt :  4105
image :  18529/18529.jpg
test1
cnt :  4106
image :  18788/18788.jpg
test1
cnt :  4107
image :  18798/18798.jpg
test1
cnt :  4108
image :  18575/18575.jpg
test1
cnt :  4109
image :  18804/18804.jpg
test1
cnt :  4110
image :  18830/18830.jpg
test1
cnt :  4111
image :  18841/18841.jpg
test1
cnt :  4112
image :  18591/18591.jpg
test1
cnt :  4113
image :  18848/18848.jpg
test1
cnt :  4114
image :  18867/18867.jpg
test1
cnt :  4115
image :  18870/18870.jpg
test1
cnt :  4116
image :  18891/18891.jpg
test1
cnt :  4117
image :  18621/18621.jpg
test1
cnt :  4118
image :  18897/18897.jpg
test1
cnt :  4119
image :  18901/18901.jpg
test1
cnt :  4120
image :  18910/18910.jpg
test1
cnt :  4121
image :  18934/18934.jpg
test1
cnt :  4122
image :  18941/18941.jpg
test1
cnt :  4123
image :  18949/18949.jpg
test1
cnt :  4124
image :  18951/18951.jpg
test1
cnt :  4125
image :  18683/186

In [ ]:
print('dataset folder : ', len(train_datasets))

no_jsons = []

cnt = 0

for i, train_data in enumerate(train_datasets):
    annot = f'{train_data}'+'/'+f'{train_data}.json'
    print('annotations : ',annot)
    if os.path.isfile(annot):
        print('test1')
        sh.copy(annot, './annotations/')
    else:
        print('test2')
        no_jsons.append(annot)
        len(no_jsons)

    cnt += 1
    print('cnt : ',cnt)

print('no_jsons_list : ',len(no_jsons))
print(no_jsons)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
annotations :  18755/18755.json
test1
cnt :  4104
annotations :  18787/18787.json
test1
cnt :  4105
annotations :  18529/18529.json
test1
cnt :  4106
annotations :  18788/18788.json
test1
cnt :  4107
annotations :  18798/18798.json
test1
cnt :  4108
annotations :  18575/18575.json
test1
cnt :  4109
annotations :  18804/18804.json
test1
cnt :  4110
annotations :  18830/18830.json
test1
cnt :  4111
annotations :  18841/18841.json
test1
cnt :  4112
annotations :  18591/18591.json
test1
cnt :  4113
annotations :  18848/18848.json
test1
cnt :  4114
annotations :  18867/18867.json
test1
cnt :  4115
annotations :  18870/18870.json
test1
cnt :  4116
annotations :  18891/18891.json
test1
cnt :  4117
annotations :  18621/18621.json
test1
cnt :  4118
annotations :  18897/18897.json
test1
cnt :  4119
annotations :  18901/18901.json
test1
cnt :  4120
annotations :  18910/18910.json
test1
cnt :  4121
annotations :  18934/18934.json
test1
cnt :  4122
annotations : 

In [34]:
!pwd
!ls
%cd /content/drive/MyDrive/aiffelthon/PjtDataset/test

/content/drive/MyDrive/aiffel/Datathon/data
'10000.csv-Grid view.csv'   json_csv.csv	    train.csv
 전체시계열_csv.csv	    sample_submission.csv   train_data_preview.xlsx
 data_backup		    test		    train.zip
 fnametest		    test.zip
 플젝데이터셋준비.ipynb     train
/content/drive/MyDrive/aiffel/Datathon/data/test


In [35]:
test_datasets = glob.glob('*')
print(test_datasets)

['66569', '66570', '66571', '66572', '66573', '66574', '66576', '66575', '66577', '66580', '66581', '66583', '66584', '66585', '66586', '66587', '66588', '66589', '66590', '66591', '66592', '66593', '66594', '66595', '66596', '66599', '66600', '66601', '66603', '66602', '66604', '66606', '66607', '66605', '66609', '66610', '66611', '66608', '66612', '66613', '66615', '66614', '66616', '66618', '66619', '66617', '66620', '66621', '66622', '66623', '66625', '66626', '66627', '66628', '66629', '66630', '66631', '66633', '66632', '66634', '66636', '66637', '66638', '66635', '66640', '66641', '66639', '66643', '66645', '66646', '66642', '66647', '66644', '66648', '66649', '66651', '66652', '66653', '66650', '66654', '66656', '66657', '66658', '66662', '66660', '66663', '66664', '66667', '66665', '66668', '66666', '66669', '66670', '66672', '66673', '66671', '66674', '66676', '66677', '66679', '66680', '66681', '66682', '66683', '66686', '66687', '66688', '66684', '66689', '66690', '66691', 

In [36]:
print('dataset folder : ', len(test_datasets))

no_images = []
no_jsones = []

cnt = 0
while (cnt < 2000):
    for i, test_data in enumerate(test_datasets):
        image = f'{test_data}'+'/'+f'{test_data}.jpg'
        print('image : ',image)
        if os.path.isfile(image):
            print('test1')
            sh.copy(image, './test_images/')
        else:
            print('test2')
            no_images.append(image)
            len(no_images)

        cnt += 1
        print('cnt : ',cnt)


print('no_images_list : ',no_images)
print('no_images_list : ',no_jsones)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
cnt :  2412
image :  64814/64814.jpg
test1
cnt :  2413
image :  64815/64815.jpg
test1
cnt :  2414
image :  64817/64817.jpg
test1
cnt :  2415
image :  64819/64819.jpg
test1
cnt :  2416
image :  64820/64820.jpg
test1
cnt :  2417
image :  64822/64822.jpg
test1
cnt :  2418
image :  64818/64818.jpg
test1
cnt :  2419
image :  64823/64823.jpg
test1
cnt :  2420
image :  64824/64824.jpg
test1
cnt :  2421
image :  64826/64826.jpg
test1
cnt :  2422
image :  64821/64821.jpg
test1
cnt :  2423
image :  64827/64827.jpg
test1
cnt :  2424
image :  64829/64829.jpg
test1
cnt :  2425
image :  64830/64830.jpg
test1
cnt :  2426
image :  64831/64831.jpg
test1
cnt :  2427
image :  64825/64825.jpg
test1
cnt :  2428
image :  64833/64833.jpg
test1
cnt :  2429
image :  64834/64834.jpg
test1
cnt :  2430
image :  64835/64835.jpg
test1
cnt :  2431
image :  64837/64837.jpg
test1
cnt :  2432
image :  64828/64828.jpg
test1
cnt :  2433
image :  64839/64839.jpg
test1
cnt :  2434
image 

KeyboardInterrupt: ignored

In [ ]:
print('dataset folder : ', len(test_datasets))

no_jsons = []

cnt = 0

for i, test_data in enumerate(test_datasets):
    annot = f'{test_data}'+'/'+f'{test_data}.json'
    print('annotations : ',annot)
    if os.path.isfile(annot):
        print('test1')
        sh.copy(annot, './test_annotations/')
    else:
        print('test2')
        no_jsons.append(annot)
        len(no_jsons)

    cnt += 1
    print('cnt : ',cnt)

print('no_jsons_list : ',len(no_jsons))
print(no_jsons)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
image :  18755/18755.jpg
test1
cnt :  4104
image :  18787/18787.jpg
test1
cnt :  4105
image :  18529/18529.jpg
test1
cnt :  4106
image :  18788/18788.jpg
test1
cnt :  4107
image :  18798/18798.jpg
test1
cnt :  4108
image :  18575/18575.jpg
test1
cnt :  4109
image :  18804/18804.jpg
test1
cnt :  4110
image :  18830/18830.jpg
test1
cnt :  4111
image :  18841/18841.jpg
test1
cnt :  4112
image :  18591/18591.jpg
test1
cnt :  4113
image :  18848/18848.jpg
test1
cnt :  4114
image :  18867/18867.jpg
test1
cnt :  4115
image :  18870/18870.jpg
test1
cnt :  4116
image :  18891/18891.jpg
test1
cnt :  4117
image :  18621/18621.jpg
test1
cnt :  4118
image :  18897/18897.jpg
test1
cnt :  4119
image :  18901/18901.jpg
test1
cnt :  4120
image :  18910/18910.jpg
test1
cnt :  4121
image :  18934/18934.jpg
test1
cnt :  4122
image :  18941/18941.jpg
test1
cnt :  4123
image :  18949/18949.jpg
test1
cnt :  4124
image :  18951/18951.jpg
test1
cnt :  4125
image :  18683/186

In [ ]:
pjt_path = '/content/drive/MyDrive/aiffelthon/PjtDataset'
flist_images = glob.glob(pjt_path+'/train/images/*')
print(flist_images)

['/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25056.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25062.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25041.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25068.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25082.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25085.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25096.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25099.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25102.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25106.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25123.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25131.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25145.jpg', '/content/drive/MyDrive/aiffelthon/PjtDataset/train/images/25149.jpg', '/con

In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/MyDrive/aiffelthon/PjtDataset

/content/drive/MyDrive/aiffel/Datathon/data


In [ ]:
! ls ./images

In [ ]:
!ls ./images

In [ ]:
!pwd

/content/drive/MyDrive/aiffel/Datathon/data


In [ ]:
%cd images

/content/drive/MyDrive/aiffel/Datathon/data/images


In [ ]:
!ls

In [ ]:
%ls

In [ ]:
import filecmp

In [ ]:
!pwd

/content/drive/MyDrive/aiffel/Datathon/data/images


In [ ]:
filecmp.dircmp(images, )

3. 주의 => 요 부분은 뒤에서 따로 해주는 걸로
* 코드 내에서 파일명의 경로 설정 시에 폴더명에 .jpg 또는 .json 
확장자를 붙여서 사용하게 만들었기 때문에, 폴더만 있고 그 안에 
jpg파일이나 json파일이 없는 경우에 파일을 못 찾는 오류가 발생함.

* 위의 과정 중에 json파일이나 jpg파일이 없는 경우는 리스트로 만들긴
했지만 이와 관련한 처리를 하지 않았기 때문에 발생되는 오류임.

=> 애초에 이 작업을 한꺼번에 진행할 수도 있고(추후에 데이터셋 작업을
새롭게 한다면...), 또는 만들어진 디렉토리 구조에서 파일이 없는 경우에 대한 처리를 해도 되는데, 일단 후자의 방법으로 처리해 준다.

        train/images 파일을 모두 읽어서 리스트 imgs_list에 저장한다.
        train/annotations 파일을 모두 읽어서 리스트 annots_list 에 저장한다

        for imgs_list로의 항목을 하나(img파일명) 읽어서
            annots_list의 항목 중에 있으면 다음으로
            annots_list의 항목 중에 없으면 삭제 

        for annots_list의 항목을 하나(annot파일명) 읽어서
            imgs_list의 항목 중에 있으면 다음으로
            imgs_list의 항목 중에 없으면 삭제


In [28]:
import os
import glob
import shutil as sh
import filecmp

In [6]:
!pwd

/content


In [3]:
%cd /content/drive/MyDrive/aiffel/Datathon/data
data_path = '/content/drive/MyDrive/aiffel/Datathon/data'

/content/drive/MyDrive/aiffel/Datathon/data


In [32]:
# f_images1 = glob.glob(data_path+'/train/images/*')
# print(f_names1[:10])

# 이미지 폴더에 있는 jpg 파일 목록을 리스트로 만든다
f_images = os.listdir(data_path+'/train/images')
print(f_images[1:10])
print(len(f_images))

# annot 폴더에 있는 json 파일 목록을 리스트로 만든다
f_annots = os.listdir(data_path+'/train/annotations')
print(f_annots[:10])
print(len(f_annots))

# jpg 파일을 하나씩 꺼내서 같은 이름의 json 파일이 있는지 확인
for img in f_images:
    print(img)
    # 같은 이름의 json파일을 만든다.
    img_name = img.split('.')
    json_file = img_name[0]+'.json'
    print(json_file)

    if os.path.isfile(data_path+'/train/annotations/'+json_file):
        continue
    else:
        os.remove(data_path+'/train/images/'+img)

# json 파일을 하나씩 꺼내서 같은 이름의 jpg 파일이 있는지 확인
for annot in f_annots:
    print(annot)
    # 같은 이름의 json파일을 만든다.
    annot_name = annot.split('.')
    jpg_file = annot_name[0]+'.json'
    print(jpg_file)

    if os.path.isfile(data_path+'/train/annotations/'+jpg_file):
        continue
    else:
        os.remove(data_path+'/train/annotations/'+annot)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
24957.json
24957.json
24942.json
24942.json
24972.json
24972.json
24958.json
24958.json
25018.json
25018.json
25019.json
25019.json
25028.json
25028.json
25040.json
25040.json
25031.json
25031.json
25049.json
25049.json
45855.json
45855.json
45938.json
45938.json
45951.json
45951.json
45959.json
45959.json
45962.json
45962.json
45980.json
45980.json
45992.json
45992.json
46004.json
46004.json
46019.json
46019.json
46043.json
46043.json
46024.json
46024.json
45969.json
45969.json
46054.json
46054.json
46056.json
46056.json
46057.json
46057.json
46063.json
46063.json
46073.json
46073.json
46077.json
46077.json
46127.json
46127.json
46111.json
46111.json
46113.json
46113.json
46141.json
46141.json
46142.json
46142.json
46143.json
46143.json
46147.json
46147.json
46148.json
46148.json
46124.json
46124.json
46155.json
46155.json
46163.json
46163.json
46234.json
46234.json
46170.json
46170.json
46204.json
46204.json
46235.json
46235.json
46243.json
46243.j

In [5]:
# f_images1 = glob.glob(data_path+'/train/images/*')
# print(f_names1[:10])

# 이미지 폴더에 있는 jpg 파일 목록을 리스트로 만든다
f_images = os.listdir(data_path+'/test/images')
print(f_images[1:10])
print(len(f_images))

# annot 폴더에 있는 json 파일 목록을 리스트로 만든다
f_annots = os.listdir(data_path+'/test/annotations')
print(f_annots[:10])
print(len(f_annots))

# jpg 파일을 하나씩 꺼내서 같은 이름의 json 파일이 있는지 확인
for img in f_images:
    print(img)
    # 같은 이름의 json파일을 만든다.
    img_name = img.split('.')
    json_file = img_name[0]+'.json'
    print(json_file)

    if os.path.isfile(data_path+'/test/annotations/'+json_file):
        continue
    else:
        os.remove(data_path+'/test/images/'+img)

# json 파일을 하나씩 꺼내서 같은 이름의 jpg 파일이 있는지 확인
for annot in f_annots:
    print(annot)
    # 같은 이름의 json파일을 만든다.
    annot_name = annot.split('.')
    jpg_file = annot_name[0]+'.json'
    print(jpg_file)

    if os.path.isfile(data_path+'/train/annotations/'+jpg_file):
        continue
    else:
        os.remove(data_path+'/train/annotations/'+annot)

/content/drive/MyDrive/aiffel/Datathon/data


In [6]:
!ls

'10000.csv-Grid view.csv'   json_csv.csv	    train.csv
 전체시계열_csv.csv	    sample_submission.csv   train_data_preview.xlsx
 data_backup		    test		    train.zip
 fnametest		    test.zip
 플젝데이터셋준비.ipynb     train


In [1]:
import time
>>> now = time.gmtime(4248.9)  
>>> now.tm_hour, now.tm_min, now.tm_sec   # 시, 분, 초

(1, 10, 48)

In [10]:
%cd /content/drive/MyDrive/aiffelthon/PjtDataset/

/content/drive/MyDrive/aiffel/Datathon/data


In [18]:
!ls

'10000.csv-Grid view.csv'		 split_annotations
 annotations				 split_images
 전체시계열_csv.csv			 test
 data_backup				 test_annotations
 disease_detection_FasterRCNN_00.ipynb	 test_images
 fnametest				 test.zip
 images					 train
 플젝데이터셋준비.ipynb			 train.csv
 json_csv.csv				 train_data_preview.xlsx
 sample_submission.csv			 train.zip


In [20]:
import splitfolders
# 이렇게 각각 나눠주면.. 랜덤하게 파일을 나누기 때문에 두 폴더의 구성이 달라지게 되므로 이렇게 사후에 하면 안됨.
data_path = '/content/drive/MyDrive/PjtDataset'
splitfolders.ratio('/content/drive/MyDrive/aiffelthon/PjtDataset/images', output='/content/drive/MyDrive/aiffelthon/PjtDataset/split_images', seed=1337, ratio=(.8,.2))
splitfolders.ratio('/content/drive/MyDrive/aiffelthon/PjtDataset/annotations', output='/content/drive/MyDrive/aiffelthon/PjtDataset/split_annotations', seed=1337, ratio=(.8,.2))

Copying files: 5745 files [00:57, 99.46 files/s] 
Copying files: 5745 files [00:59, 96.10 files/s] 
